In [1]:
#comment cell

# Uh-oh! The CEO wants more info (he's really needy). The CEO wants to know the following (for your regional store data only):

# If a customer buys milk, does that imply the customer will buy cereal? And how confident are we in that?
# If a customer does NOT buy milk, does that imply the customer will buy cereal? And how confident are we in that?
# If a customer buys baby food, does that imply the customer will buy diapers? And how confident are we in that?
# If a customer buys peanut butter, does that imply the customer will buy jelly or jam? And how confident are we in that?
# If a customer does NOT buy peanut butter, does that imply the customer will buy jelly or jam? And how confident are we in that?
# Sounds like a market basket analysis problem if I've ever heard one! Use the excel file in the homework module to get the CEO off your back!

In [1]:
#import statments to use

import sqlite3
import csv 
import mlxtend
import pandas as pd
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
import pandas as pd

In [3]:
#first lets create the database and tables

# Connect to SQLite database (or create if it doesn't exist)
conn = sqlite3.connect('analysis.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Define SQL query to create table
create_table_query = """
CREATE TABLE IF NOT EXISTS analysis (
    last_name TEXT,
    first_name TEXT,
    cust_id INTEGER
);
"""

# Execute the SQL query to create the table
cursor.execute(create_table_query)

# Read data from existing CSV file
with open('southcentral.csv', 'r') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    for row in csv_reader:
        # Insert data into SQLite table
        cursor.execute("INSERT INTO analysis VALUES (?, ?, ?)", (row['last_name'], row['first_name'], int(row['cust_id'])))

# Commit changes and close connection
conn.commit()
conn.close()

In [4]:
# lets create the jan_southcentral_purchases.csv file
#first lets create the database and tables
# Connect to SQLite database (or create if it doesn't exist)
conn = sqlite3.connect('analysis.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Define SQL query to create table
create_table_query = """
CREATE TABLE IF NOT EXISTS southcentral_purchases (
    SKU INTEGER,
    SalePrice INTEGER
);
"""

# Execute the SQL query to create the table
cursor.execute(create_table_query)

# Read data from existing CSV file
with open('2024_jan_southcentral_purchases.csv', 'r') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    for row in csv_reader:
        # Insert data into SQLite table
        cursor.execute("INSERT INTO southcentral_purchases VALUES (?, ?)", (row['SKU'], row['SalePrice']))

# Commit changes and close connection
conn.commit()
conn.close()

In [9]:
import sqlite3
import csv

# Connect to SQLite database (or create if it doesn't exist)
conn = sqlite3.connect('analysis.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Define SQL query to create table
create_table_query = """
CREATE TABLE IF NOT EXISTS products (
    Manufacturer TEXT,
    ProductName TEXT,
    Size TEXT,
    ItemType TEXT,
    SKU INTEGER,
    BasePrice REAL
);
"""

# Execute the SQL query to create the table
cursor.execute(create_table_query)

# Read data from existing CSV file and insert into table
with open('products.csv', 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip header row
    for row in csv_reader:
        cursor.execute("INSERT INTO products VALUES (?, ?, ?, ?, ?, ?)", row)

# Commit changes and close connection
conn.commit()
conn.close()


In [3]:
# Read transaction data from CSV
data = pd.read_csv('products.csv')
# Perform one-hot encoding to convert categorical variables into binary indicators
binary_data = pd.get_dummies(data)

# Convert all non-zero values to 1 to represent presence
binary_data[binary_data != 0] = 1

# Optionally, you can drop any duplicate rows
binary_data = binary_data.drop_duplicates()


C:\Users\David\AppData\Local\Temp\ipykernel_13996\2764200154.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  binary_data[binary_data != 0] = 1


In [4]:
import pandas as pd

# Load the first CSV file containing product information
products_data = pd.read_csv('products.csv')

# Load the second CSV file containing SKU and sale price
sales_data = pd.read_csv('2024_jan_southcentral_purchases.csv')

# Aggregate sales data by SKU
sales_data_aggregated = sales_data.groupby('SKU').agg({'SalePrice': 'mean'}).reset_index()

# Merge the aggregated sales data with the products data
merged_data = pd.merge(products_data, sales_data_aggregated, on='SKU', how='inner')




# Merge the two dataframes on the SKU column
merged_data = pd.merge(products_data, sales_data, on='SKU', how='inner')

# Display the merged data
print(merged_data.head())

merged_data.to_csv('merged_data.csv', index=False)


  Manufacturer        Product Name   Size       itemType       SKU BasePrice  \
0    Zatarains  Jambalaya Rice Mix  12 oz  Rice/Rice Mix  42081001     $2.49   
1    Zatarains  Jambalaya Rice Mix  12 oz  Rice/Rice Mix  42081001     $2.49   
2    Zatarains  Jambalaya Rice Mix  12 oz  Rice/Rice Mix  42081001     $2.49   
3    Zatarains  Jambalaya Rice Mix  12 oz  Rice/Rice Mix  42081001     $2.49   
4    Zatarains  Jambalaya Rice Mix  12 oz  Rice/Rice Mix  42081001     $2.49   

   SalePrice  
0       3.74  
1       3.74  
2       3.74  
3       3.74  
4       3.74  


In [5]:
# Load the merged data from the CSV file
merged_data = pd.read_csv('merged_data.csv')

# Remove duplicate rows
merged_data_no_duplicates = merged_data.drop_duplicates()

# Save the DataFrame with no duplicates to a new CSV file
merged_data_no_duplicates.to_csv('merged_data_no_duplicates.csv', index=False)


In [4]:
from mlxtend.frequent_patterns import association_rules, apriori
from mlxtend.preprocessing import TransactionEncoder

# Load the merged data from the CSV file
merged_data = pd.read_csv('merged_data_no_duplicates.csv')

# Pivot the data to create a transactional format
transactions = merged_data.pivot_table(index=['SKU', 'SalePrice'], columns='Product Name', aggfunc='size', fill_value=0)

# Convert index to strings
transactions.index = transactions.index.map(lambda x: '_'.join(map(str, x)))

# Apply one-hot encoding to the data
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# Apply Apriori algorithm to find frequent itemsets
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)

# Define function to get association rules
def get_association_rules(itemset, metric='confidence', min_threshold=0.5):
    rules = association_rules(itemset, metric=metric, min_threshold=min_threshold)
    return rules

# Function to check association rules for given products
def check_association(item1, item2):
    rules = get_association_rules(frequent_itemsets)
    for _, rule in rules.iterrows():
        if (rule["antecedents"] == frozenset({item1})) and (rule["consequents"] == frozenset({item2})):
            return True, rule["confidence"]
    return False, None

# Question 1: If a customer buys milk, does that imply the customer will buy cereal?
milk_to_cereal, confidence1 = check_association('Milk', 'Cereal')
print("If a customer buys milk, does that imply the customer will buy cereal?", milk_to_cereal)
if milk_to_cereal:
    print("Confidence:", confidence1)

# Question 2: If a customer does NOT buy milk, does that imply the customer will buy cereal?
no_milk_to_cereal, confidence2 = check_association('Milk', 'Cereal')
print("If a customer does NOT buy milk, does that imply the customer will buy cereal?", not no_milk_to_cereal)
if not no_milk_to_cereal:
    print("Confidence:", 1 - confidence2)

# Question 3: If a customer buys baby food, does that imply the customer will buy diapers?
baby_food_to_diapers, confidence3 = check_association('Baby Food', 'Diapers')
print("If a customer buys baby food, does that imply the customer will buy diapers?", baby_food_to_diapers)
if baby_food_to_diapers:
    print("Confidence:", confidence3)

# Question 4: If a customer buys peanut butter, does that imply the customer will buy jelly or jam?
peanut_butter_to_jelly, confidence4 = check_association('Peanut Butter', 'Jelly')
print("If a customer buys peanut butter, does that imply the customer will buy jelly?", peanut_butter_to_jelly)
if peanut_butter_to_jelly:
    print("Confidence:", confidence4)

# Question 5: If a customer does NOT buy peanut butter, does that imply the customer will buy jelly or jam?
no_peanut_butter_to_jelly, confidence5 = check_association('Peanut Butter', 'Jelly')
print("If a customer does NOT buy peanut butter, does that imply the customer will buy jelly?", not no_peanut_butter_to_jelly)
if not no_peanut_butter_to_jelly:
    print("Confidence:", 1 - confidence5)


If a customer buys milk, does that imply the customer will buy cereal? False


KeyboardInterrupt: 